# **Inicializar**


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import seaborn as sns
# carregando as informações do df
#insira aqui o path dos arquivos de vinhos tintos e brancos
vinho_red = pd.read_csv(r"C:\Users\jluca\OneDrive\Documentos\UFC\6º semestre\Inteligência Computacional Aplicada\HW 1\winequality-red.csv",delimiter=';')
vinho_white = pd.read_csv(r"C:\Users\jluca\OneDrive\Documentos\UFC\6º semestre\Inteligência Computacional Aplicada\HW 1\winequality-white.csv",delimiter=';')

# **Plot da da análise monovariada do histograma, box-plot e tabela com média, variança e skewness&**

## Código inicial

In [ ]:
def plotar_histograma_box(data_frame,vinho,cor):
  for i in data_frame:
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
    # Plota o histograma
    ax1.hist(data_frame[i], bins=20,color=cor, edgecolor='black')
    ax1.set_title(f'Hist. de freq. de {i} para vinhos {vinho}')
    ax1.set_xlabel(f'Valores de {i}')
    ax1.set_ylabel('Frequência')
    # Plota o box-plot
    ax2.boxplot(data_frame[i], vert=True)
    ax2.set_title(f'Box-Plot {i} para vinhos tinto')
    plt.tight_layout()
    plt.show()


In [ ]:
def calcular_tabela(data_frame):
  colunas = ['mean', 'standard deviation', 'skewness']
  linhas = [ 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
      'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
      'pH', 'sulphates', 'alcohol', 'quality']
  listas = []
  # Calcula a média, desvio padrão e skewness para cada preditor e para quality
  for i in data_frame:
    listas.append(data_frame[i].mean())
    listas.append(data_frame[i].std())
    listas.append(data_frame[i].skew())

  lista = np.array([listas])
  matriz = lista.reshape(12, 3)
  tabela = pd.DataFrame(data=matriz,index=linhas,columns=colunas)
  return tabela


## Vinhos tintos

In [ ]:
plotar_histograma_box(vinho_red,'tinto','maroon')

## Tabela Vinhos tintos

In [ ]:
tabela = calcular_tabela(vinho_red)
print("\t\t\tVinhos tintos")
tabela

## Vinhos brancos

In [ ]:
plotar_histograma_box(vinho_white,'branco', 'aliceblue')

## Tabela Vinhos Brancos

In [ ]:
tabela = calcular_tabela(vinho_white)
print("\t\t\tVinhos brancos")
tabela

# **Plot da análise monovariada condicionada a classe do histograma, box-plot e tabela com média, variança e skewness**

## Código inicial

In [ ]:
def calcular_tabela_conditional(dataframe):
  # Definindo a coluna da classe
  classe_coluna = 'quality'
  # Pegando os elementos da classe
  classes_unicas = dataframe[classe_coluna].unique()
  # Usado para salvar as informações estatísticas de cada class-conditional mono-variate analysis
  dicionario = {}
  colunas = ['mean', 'standard deviation', 'skewness']
  linha = [ 'fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
      'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
      'pH', 'sulphates', 'alcohol']

  for classe in classes_unicas:
      # Filtrar os dados para a classe atual
      dados_classe = dataframe[dataframe[classe_coluna] == classe]
      listas = []
      # Realiza o calculo da média, desvio padrão e skewness
      for coluna in dataframe.columns:
          if coluna != classe_coluna:  # Exclui a coluna da classe
              listas.append(dados_classe[coluna].mean())
              listas.append(dados_classe[coluna].std())
              listas.append(dados_classe[coluna].skew())

      dicionario.update({ f'Classe {classe}' : listas})

  pd.set_option('display.max_columns', None)
  pd.set_option('display.max_rows', None)

  # plotando a tabela
  for chave in dicionario.keys():
    lista = np.array([dicionario[chave]])
    print(f'Tabela da classe {chave}')
    matriz = lista.reshape(11, 3)
    tabela = pd.DataFrame(data=matriz,index=linha,columns=colunas)
    display(tabela)
    print('\n\n')


In [ ]:
def classe_conditional(dataframe,vinho,cor):
  classe_coluna = 'quality'
  classes_unicas = dataframe[classe_coluna].unique()

  for classe in classes_unicas:
      # Filtrar os dados para a classe atual
      dados_classe = dataframe[dataframe[classe_coluna] == classe]
      for coluna in dataframe.columns:
          if coluna != classe_coluna:  # Exclua a coluna da classe
              # Plote histograma e box-plot para a class-conditional mono-variate analysis
              fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

              ax1.hist(dados_classe[coluna], bins=20,color=cor, edgecolor='black')
              ax1.set_title(f'Hist de freq. de {coluna} para vinhos {vinho} (Classe {classe})')
              ax1.set_xlabel(f'Valores de {coluna}')
              ax1.set_ylabel('Frequência')

              ax2.boxplot(dados_classe[coluna], vert=True)
              ax2.set_title(f'Box-Plot {coluna} para vinhos tinto (Classe {classe})')
              plt.tight_layout()
              plt.show()


## Vinho Tinto

In [ ]:
classe_conditional(vinho_red,'tinto','maroon')

## Tabela Vinho Tinto

In [ ]:
calcular_tabela_conditional(vinho_red)

## Vinho Branco

In [ ]:
classe_conditional(vinho_white,'branco','aliceblue')

## Tabela Vinho Branco

In [ ]:
calcular_tabela_conditional(vinho_white)

# **Plot da análise bivariada e da matriz de correlação**

## Código inicial

In [ ]:
def bivariante(dataframe,vinho):
  # filtro de coluna para separar os preditoras da classe
  classe_coluna = 'quality'
  for i in dataframe:
    cont = 1
    corr = []
    if i!=classe_coluna:
      fig = plt.figure(figsize=(15,15))
      for j in dataframe:
        if j!=classe_coluna:
          fig.add_subplot(4,3,cont)
          # Calcula a reta de regressão
          m, b = np.polyfit(dataframe[i], dataframe[j], 1)
          # Plota os pontos
          plt.scatter(dataframe[i], dataframe[j], c=dataframe[classe_coluna], cmap='viridis')
          plt.plot(dataframe[i], m*dataframe[i] + b)
          plt.xlabel(f'{i} do vinho {vinho}')
          plt.ylabel(f'{j}')
          cont += 1

    plt.suptitle(f'Scatter Plots entre Preditores do vinho {vinho}')
    plt.tight_layout()
    plt.show()

In [ ]:
def matrix_correlacao(dataframe,vinho):
  classe_coluna = 'quality'
  correlation_matrix = []
  linha = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
      'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
      'pH', 'sulphates', 'alcohol']

  for i in dataframe:
    cont = 1
    corr = []
    if i!=classe_coluna:
      for j in dataframe:
        if j!=classe_coluna:
          # Calcula a correlação entre dois preditores
          corr.append(dataframe[[i, j]].corr().iloc[0, 1])
      correlation_matrix.append(corr)
  x_axis_labels = linha
  y_axis_labels = linha

  plt.figure(figsize=(10,10))
  # Plota a matriz de correlação
  ax = sns.heatmap(correlation_matrix, xticklabels=x_axis_labels, yticklabels=y_axis_labels , annot=True, cmap="coolwarm", fmt=".2f")
  plt.title(f'Matrix de Correlação do vinho {vinho}')

## Vinhos tintos

In [ ]:
bivariante(vinho_red,'tinto')

## Matriz de Correlação de Vinhos tintos

In [ ]:
matrix_correlacao(vinho_red,'tinto')

## Vinhos brancos

In [ ]:
bivariante(vinho_white,'branco')

## Matriz de Correlação de Vinhos brancos

In [ ]:
matrix_correlacao(vinho_white,'branco')

# **PCA e screen plot**

## Código

In [ ]:
# Cria uma cópia do data_set somente com os preditores
def screen(dataframe,vinho):
  df = dataframe.copy(deep=True)
  x = df.drop('quality',axis=1)
  y = dataframe['quality']

  # Normalização dos dados e aplicação do PCA
  scaler = StandardScaler()
  X = scaler.fit_transform(x)
  pca = PCA(n_components = 2)
  principal_components = pca.fit_transform(X)

  # Plot do screen plot
  new_X = pd.DataFrame(data = principal_components, columns = ['PC1', 'PC2'])
  per_var = np.round(pca.explained_variance_ratio_ * 100, decimals=1)
  label = ['PC' + str(x) for x in range(1, len(per_var)+1)]
  plt.bar(x=range(1, len(per_var)+1), height = per_var, tick_label = label)

  plt.ylabel('porcentangem de variação')
  plt.xlabel('Principal components')
  plt.title(f'screen plot para vinhos {vinho}')
  plt.show()

In [ ]:
def aplicar_PCA(dataframe,vinho):
  df = dataframe.copy(deep=True)
  df = df.drop('quality',axis=1)

  scaling=StandardScaler(with_mean=True,with_std=True)
  scaling.fit(df)
  Scaled_data=scaling.transform(df)

  principal=PCA(n_components=11)
  principal.fit(Scaled_data)

  x=principal.transform(Scaled_data)
  # plot do PCA
  plt.figure(figsize=(10,10))
  plt.scatter(x[:,0],x[:,1],c=dataframe['quality'], cmap='viridis')
  plt.xlabel('pc1')
  plt.ylabel('pc2')
  plt.title(f'PCA plot para vinhos {vinho}')
  eigenvalues = principal.explained_variance_ratio_


## Vinhos Tintos

In [ ]:
screen(vinho_red,'tintos')

In [ ]:
aplicar_PCA(vinho_red,'tintos')

## Vinhos Brancos

In [ ]:
screen(vinho_white,'brancos')

In [ ]:
aplicar_PCA(vinho_white,'brancos')